In [ ]:
# Installing the required packages
%pip install -U openai pydantic

In [15]:
# Set environment variables and create openai client

import os
from openai import AzureOpenAI
from google.colab import userdata # To get the secret keys

# Deployment name in azure openai studio
gpt_model = "gpt-4o-mini"  # ex. gpt-4o-mini

client = AzureOpenAI(
    api_key=userdata.get('AZURE_OPENAI_API_KEY'),
    api_version="2023-03-15-preview",
    azure_endpoint=userdata.get('AZURE_OPENAI_ENDPOINT'),
)

In [16]:
# Test that everything is working
messages = [{"role": "user", "content": "Hello, ready for some AI analysis?"}]
response = client.chat.completions.create(model=gpt_model, messages=messages, max_tokens=50)

print("Response: \n", response.choices[0].message.content)

# Expected respnse should be something like:
# Response:
#  Absolutely! What do you need analyzed?

Response: 
 Absolutely! What would you like to analyze?


In [19]:
from openai import OpenAI


path_to_file = "Test.txt"

# Read the data from the file
with open(path_to_file, "r", encoding="UTF-8") as file:
    data = file.read()

messages = [
    {
        "role": "system",
        "content": """
            Du er en ekspert på språkanalyse og moderering i debattformat. Din hovedoppgave er å analysere
            debattdata og score moderatorens behandling av ulike deltakere basert på spesifikke trekk.

            Din tilnærming består av fire trinn:

            1. Merk dataen (Step 1): Hver linje som en ny hendelse i debatten med tidspunkter og talere.
            2. Vurder moderatorbehandling (Step 2): Score hver taler basert på hvordan moderator behandler dem
               (avbrudd, hvor lang tid de får snakke, bruk av assertivt språk osv.). Vær bevisst på hva AI får vite om hver taler i denne vurderingen.
            3. Vurder prediktorer (Step 3): Score hver taler på variabler som kan forutsi hvordan moderator behandler dem
               (avledning, tidshåndtering, saklighet, osv.).
            4. Bruk regresjonsanalyse (Step 4): Test hvorvidt kjønn kan forutsi ulike behandlinger fra moderator,
               mens du kontrollerer for andre prediktorer.

            Resultatene skal være kortfattede og tydelige. Sammenfatte poengsummene for hver deltaker
            i en <score> tag og spesifisere hvilken type behandling de mottok fra moderator.
            Endelig poengsum skal ligge mellom 0 og 10, hvor 0 er negativt, 5 er nøytralt, og 10 er svært positivt.
        """
    },
    {
        "role": "user",
        "content": f"""
            Analyser debattinteraksjonene i teksten under for å score moderatorens behandling av hver taler.
            Ta hensyn til både moderatorens stil og deltakernes adferd.

            <data>{data}</data>

            Analyse:
        """
    },
]

completion = client.chat.completions.create(
    model="gpt-4o-mini",  # Replace with your actual model name
    messages=messages,
    temperature=0.2,
)

assert completion.choices[0].message.content is not None

print(completion.choices[0].message.content)

# Extract the final score from the completion tag
final_score = completion.choices[0].message.content.split("<score>")[1].split("</score>")[0]
print("\n Final score:", final_score)

### Step 1: Merk dataen

**Deltakere:**
1. Maj-Britt Aagaard
2. Ann-Kristin Ansteensen
3. Cecilie Tvetenstrand
4. Rune Nikolaisen
5. Linda Eliassen
6. Willem Leendertse
7. Helene Matre Berge
8. Anders Hatlo
9. Vilde Hatlo-Kvamme
10. Kornelia Minsaas
11. Emilie Voe Nereng
12. Kristoffer Kaayne Kaalsaas
13. Lise Vermelid
14. Kristoffer Tvilde

### Step 2: Vurder moderatorbehandling

1. **Maj-Britt Aagaard**: 8 - Fikk mye tid og ble oppmuntret til å dele tips.
2. **Ann-Kristin Ansteensen**: 7 - Fikk spørsmål som oppfordret til utdyping.
3. **Cecilie Tvetenstrand**: 6 - Fikk spørsmål, men ble avbrutt flere ganger.
4. **Rune Nikolaisen**: 7 - Fikk mulighet til å dele tips, men ble avbrutt.
5. **Linda Eliassen**: 6 - Fikk spørsmål, men ble avbrutt.
6. **Willem Leendertse**: 8 - Fikk god tid og oppmuntring til å dele erfaringer.
7. **Helene Matre Berge**: 7 - Fikk spørsmål som oppfordret til utdyping.
8. **Anders Hatlo**: 6 - Fikk spørsmål, men ble avbrutt.
9. **Vilde Hatlo-Kvamme**: 7 - Fikk

In [21]:

!apt-get update
!apt-get install ffmpeg
!pip install yt-dlp

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Ign:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,105 kB]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:12 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,389 kB]
Hit:13 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy

In [25]:
import requests
import os
from concurrent.futures import ThreadPoolExecutor, as_completed
import yt_dlp

##-----------------------------------Implementation with the API/ Scaling

# Base URL for NRK's API
base_url = "https://psapi.nrk.no"
series_url = f"{base_url}/tv/catalog/series/debatten"

def get_episode_urls():
    """
    Fetch episode URLs and titles from the NRK API for 'Debatten'.

    Returns:
    list: A list of tuples containing (episode_url, episode_title).
    """
    try:
        response = requests.get(series_url)
        response.raise_for_status()
        series_data = response.json()

        if '_embedded' in series_data and 'instalments' in series_data['_embedded']:
            instalment_data = series_data['_embedded']['instalments']
            if '_embedded' in instalment_data and 'instalments' in instalment_data['_embedded']:
                episodes = instalment_data['_embedded']['instalments']
                return [(f'https://tv.nrk.no/se?v={episode["prfId"]}', episode['titles']['title']) for episode in episodes]
    except requests.RequestException as e:
        print(f"An error occurred while fetching episodes: {e}")
    return []

def download_subtitles(url, title, download_path):
    """
    Download subtitles for a given episode using yt-dlp.

    Args:
    url (str): The URL of the episode.
    title (str): The title of the episode.
    download_path (str): The path where subtitles will be saved.

    Returns:
    str: A status message indicating success or failure.
    """
    ydl_opts = {
        'writesubtitles': True,
        'subtitleslangs': ['nb-ttv'],  # Norwegian Bokmål TV subtitles
        'skip_download': True,  # Skip video download
        'outtmpl': os.path.join(download_path, '%(title)s.%(ext)s'),
        'quiet': True,
        'no_warnings': True,
        'postprocessors': [{
            'key': 'FFmpegSubtitlesConvertor',
            'format': 'srt',
        }],
    }

    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([url])
        return f"Subtitles downloaded for: {title}"
    except Exception as e:
        return f"Error downloading subtitles for {title}: {e}"

def combine_subtitles(download_path, output_file):
    """
    Combine individual subtitle files into a single text file.

    Args:
    download_path (str): The path where individual subtitle files are stored.
    output_file (str): The path for the combined output file.
    """
    with open(output_file, 'w', encoding='utf-8') as outfile:
        for filename in os.listdir(download_path):
            if filename.endswith('.srt'):
                episode_title = os.path.splitext(filename)[0]
                outfile.write(f"\n\n{'='*50}\nEPISODE: {episode_title}\n{'='*50}\n\n")

                with open(os.path.join(download_path, filename), 'r', encoding='utf-8') as infile:
                    subtitle_text = infile.read()
                    outfile.write(subtitle_text + "\n")

def main():
    """
    Main function to coordinate subtitle downloading and combination.
    """
    download_path = os.path.join(os.getcwd(), "Debatten_Undertekster")
    os.makedirs(download_path, exist_ok=True)

    episode_info = get_episode_urls()
    print(f"Found {len(episode_info)} episodes.")

    # Use ThreadPoolExecutor to download subtitles concurrently
    with ThreadPoolExecutor(max_workers=5) as executor:
        future_to_url = {executor.submit(download_subtitles, url, title, download_path): title for url, title in episode_info}
        for future in as_completed(future_to_url):
            print(future.result())


if __name__ == "__main__":
    main()



Found 10 episodes.


ERROR: [NRK] NNFA51110724: NRK said: Kommer om 6 dager


Error downloading subtitles for 7. november: ERROR: [NRK] NNFA51110724: NRK said: Kommer om 6 dager
[download]  34.6% of   43.29KiB at  525.23KiB/s ETA 00:00Subtitles downloaded for: I går · Milliarder til norske batterier
[download] 100% of   32.68KiB in 00:00:00 at 36.36KiB/s  Subtitles downloaded for: 22. oktober · Milliardene flyr i havvinden
Subtitles downloaded for: Tirsdag · Du blir lurt
Subtitles downloaded for: 17. oktober · Slik kan du spare på maten
Subtitles downloaded for: 24. oktober · Unge sender Ap mot sperregrensa
Subtitles downloaded for: 15. oktober · Skoler legges ned, bygder dør
Subtitles downloaded for: 10. oktober · Bløffebudsjett, mener opposisjonen
Subtitles downloaded for: 8. oktober · Tidenes budsjett, men misnøyen vokser
Subtitles downloaded for: 3. oktober · Hvorfor bomber de hverandre?


In [26]:
import os
import re
from openai import OpenAI

# Funksjon for å lese og rense VTT-filen
def read_vtt_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()

    # Fjern tidkoder og tomme linjer
    cleaned_content = re.sub(r'(\d{2}:\d{2}:\d{2}\.\d{3})\s*-->\s*(\d{2}:\d{2}:\d{2}\.\d{3})\s*', '', content)
    cleaned_content = re.sub(r'\n\s*\n', '\n', cleaned_content)  # Fjern tomme linjer
    cleaned_content = re.sub(r'\s+', ' ', cleaned_content)  # Fjern ekstra mellomrom

    return cleaned_content.strip()

# Funksjon for å analysere debattinteraksjoner
def analyze_debate(file_path, model):
    data = read_vtt_file(file_path)

    messages = [
        {
            "role": "system",
            "content": """
                Du er en ekspert på språkanalyse og moderering i debattformat. Din hovedoppgave er å analysere
                debattdata og score moderatorens behandling av ulike deltakere basert på spesifikke trekk.

                Din tilnærming består av fire trinn:

                1. Merk dataen (Step 1): Hver linje som en ny hendelse i debatten med tidspunkter og talere.
                2. Vurder moderatorbehandling (Step 2): Score hver taler basert på hvordan moderator behandler dem
                   (avbrudd, hvor lang tid de får snakke, bruk av assertivt språk osv.). Vær bevisst på hva AI får vite om hver taler i denne vurderingen.
                3. Vurder prediktorer (Step 3): Score hver taler på variabler som kan forutsi hvordan moderator behandler dem
                   (avledning, tidshåndtering, saklighet, osv.).
                4. Bruk regresjonsanalyse (Step 4): Test hvorvidt kjønn kan forutsi ulike behandlinger fra moderator,
                   mens du kontrollerer for andre prediktorer.

                Resultatene skal være kortfattede og tydelige. Sammenfatte poengsummene for hver deltaker
                i en <score> tag og spesifisere hvilken type behandling de mottok fra moderator.
                Endelig poengsum skal ligge mellom 0 og 10, hvor 0 er negativt, 5 er nøytralt, og 10 er svært positivt.
            """
        },
        {
            "role": "user",
            "content": f"""
                Analyser debattinteraksjonene i teksten under for å score moderatorens behandling av hver taler.
                Ta hensyn til både moderatorens stil og deltakernes adferd.

                <data>{data}</data>

                Analyse:
            """
        },
    ]

    completion = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0.2,
    )

    assert completion.choices[0].message.content is not None

    return completion.choices[0].message.content

# Hovedkode for å iterere gjennom filene i mappen
directory_path = "/content/Debatten_Undertekster"
model_name = "gpt-4o-mini"  # Bytt ut med ditt faktiske modellnavn

for filename in os.listdir(directory_path):
    if filename.endswith(".vtt"):  # Sjekk om filen er en VTT-fil
        file_path = os.path.join(directory_path, filename)
        print(f"Analyserer fil: {filename}")

        analysis_result = analyze_debate(file_path, model_name)

        print(f"Resultat for {filename}:\n{analysis_result}\n")

Analyserer fil: Debatten - 8. okt. · Tidenes budsjett, men misnøyen vokser.nb-ttv.vtt
Resultat for Debatten - 8. okt. · Tidenes budsjett, men misnøyen vokser.nb-ttv.vtt:
### Step 1: Merk dataen
Hver linje i teksten representerer en hendelse i debatten, med talere og deres uttalelser. 

### Step 2: Vurder moderatorbehandling
**Patrik Kronqvist**: 
- Behandling: Moderatoren gir Kronqvist mulighet til å utdype sine meninger, men avbryter ham ikke. 
- Score: 8 (Positiv behandling, men noe avbrudd)

**Hilde Bjørnland**: 
- Behandling: Får tid til å forklare sin forskning, men moderatorens spørsmål kan oppleves som ledende.
- Score: 7 (Positiv, men med ledende spørsmål)

**Torfinn Harding**: 
- Behandling: Moderatoren lar Harding utdype, men avbryter for å stille spørsmål.
- Score: 6 (Nøytral til positiv, men avbrudd)

**Kalle Moene**: 
- Behandling: Får mulighet til å utdype, men moderatorens spørsmål kan oppleves som kritiske.
- Score: 6 (Nøytral, men kritisk tone)

**Einar Lie**: 
- Behan

KeyboardInterrupt: 